# Factors.jl
A package for factors/potentials with parametric supports

In [1]:
using DataFrames
using Factors

## Dimensions

The core unit are dimensions, which consist of names (`Symbol`) and a countably-finite supports (`<: AbstractVector`):

In [2]:
map(s -> Dimension(:X, s), [["bob", "waldo", "superman"], ('a', 'α'), 'a':2:'z', 10:3:40, 2:15, 1:4, 16])

7-element Array{Any,1}:
 X:  String["bob","waldo","superman"] (3)
 X:  ['a','α'] (2)                       
 X:  'a':2:'y' (13)                      
 X:  10:3:40 (11)                        
 X:  2:15 (14)                           
 X:  1:4                                 
 X:  1:16                                

### Indexing and iterating

In [3]:
x = Dimension(:X, 'α':'ω')

for v in x
    print(v, " ")
end

α β γ δ ε ζ η θ ι κ λ μ ν ξ ο π ρ ς σ τ υ φ χ ψ ω 

In [4]:
x[2]

'β'

In [5]:
indexin('β', x)

2

In [6]:
(I, d) = update(x, ['α', 'ψ', 'ζ', 'δ'])
I

4-element Array{Int64,1}:
  1
 24
  6
  4

In [7]:
d

X:  ['α','ψ','ζ','δ'] (4)

### Dimension Comparisons

Equality for dimensions is by state values, not by type

In [8]:
Dimension(:X, [1, 2, 3]) == Dimension(:X, 1:1:3)  == Dimension(:X, 1:3) == Dimension(:X, 3)

true

Comparisons use the position of elements in a dimension

In [9]:
o = Dimension(:X, [3, 16, -2])
o .< -2 # here, 3 & 16 are less than -2

3-element BitArray{1}:
  true
  true
 false

In [10]:
# 3, 16, and -2 are all ≥ 3
o .≥ 3

3-element BitArray{1}:
 true
 true
 true

## Factors

A factor maps from the support of each of its dimensions to a `Float64`.

Potentials are read in column-major order: the first dimension corresponds to the first axis (column), the second dimension corresponds to the second axis (rows) etc ...

In [11]:
ft = Factor([:X, :Y], [1 4; 2 5; 3 6])
DataFrame(ft)

,X,Y,potential
1,1,1,1.0
2,2,1,2.0
3,3,1,3.0
4,1,2,4.0
5,2,2,5.0
6,3,2,6.0


There are multiple convience constructors:

In [12]:
c = Dimension(:C, 3)

s = Dimension(:S, 10:2:18)

Factor([c, s], rand(3, 5))
Factor(c, [2, 0, 16])
Factor([1 4; 2 5; 3 6], :X => 3:5, :Y => ['a', 'b'])

# creates CartesianDimensions
Factor([:X, :Y], rand(20, 16)) 
Factor(:X, [31, 33, 58])

# zero potentials
Factor(c)
Factor([c, s])
Factor(Dict(:X=>14, :Y=>['Γ', 'Δ'], :Z =>'a':2:'z'))
Factor(:A=>10, :B=>3:20)

# zero dimensional factor
Factor(2016)

1 instantiation: 2016.0

### Patterns

`pattern` returns the sequence of a dimension in a factor

In [13]:
c = Dimension(:C, 3)
s = Dimension(:S, 'a':2:'h')
ft = Factor([c, s])

pattern(ft)

12×2 Array{Int64,2}:
 1  1
 2  1
 3  1
 1  2
 2  2
 3  2
 1  3
 2  3
 3  3
 1  4
 2  4
 3  4

Also done via accessing that dimension

In [14]:
pattern_states(ft)

12×2 Array{Any,2}:
 1  'a'
 2  'a'
 3  'a'
 1  'c'
 2  'c'
 3  'c'
 1  'e'
 2  'e'
 3  'e'
 1  'g'
 2  'g'
 3  'g'

A Factors scope returns the names of the dimension. As does `names(ϕ)`

In [15]:
scope(ft)

2-element Array{Factors.Dimension,1}:
 C:  1:3          
 S:  'a':2:'g' (4)

### Accessing

Iterating over a factor is done by dimensions

An `Assignment` can also select from the factor:

### Mapping

### Broadcasting

Operations can be broadcast along dimensions:

DataFrame(broadcast(*, ft, :B, [100, 0.01]))

Dimensions can be reduced.
Convience functions are provded for the following (As well as for their excited cousins, e.g. sum!):
* `sum`  
* `prod`  
* `maximum`  
* `minimum`  

### Joining

Factors can be joined through `join` or by multiplying (adding, etc.) them: